# Geometric-QML : Tic-Tac-Toe

This notebook shows an implementation of the Tic-Tac-Toe game using G-QML. 

First, import necessary libraries.

In [ ]:
import csv
import numpy as np

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms.optimizers import ADAM

import gqml_helpers as helpers

## Load data

Load **tic-tac-toe.csv**. 

Reference : Aha,David. (1991). Tic-Tac-Toe Endgame. UCI Machine Learning Repository. https://doi.org/10.24432/C5688J.

In [ ]:
with open('tic-tac-toe_csv.csv', newline = '') as f:
    reader = csv.reader(f)
    data = list(reader)

## Process the data

Flatten board positions into numerical vectors 

x present -> 1

empty square -> 0

o present -> -1

In [ ]:
encoding = {'x': 1, 'b': 0, 'o': -1}
positions = np.zeros([len(data[1:]),9])
for i in range(1,len(data)):
    for j in range(9):
        positions[i-1,j] = encoding[data[i][j]]

### Identifying labels
The original dataset contains two classes: X won or X did not win.

We separate the data into three classes: win for X ([1, -1, -1]), draw ([-1, 1, -1]), or win for O ([-1, -1, 1])

In [ ]:
labels = []
for i in range(len(positions)):
    labels.append(np.array(helpers.who_won(positions[i])))

print(labels)


### Generating train and test data 

In [ ]:
train_pos, train_labels, test_pos, test_labels = helpers.split_data(0.2, positions, labels)

Setup Parameters dictionary for later use in quantum circuit. 

In [ ]:
data_param_dict = {}
for i in range(9):
    data_param_dict[f'x_{i}'] = Parameter(f'x_{i}')

## Define loss function

The loss function runs over all elements in the training set and calculates the L2 norm between the prediction and target.


In [ ]:
def loss(qc, train_pos, train_classes, theta_list):
    loss_val = 0
    for i, pos in enumerate(train_pos):
        to_bind = np.concatenate((theta_list, pos), axis=0)
        qc_temp = qc.bind_parameters(to_bind)
        label = np.array([helpers.measure_z_corners(qc_temp), helpers.measure_z_mid(qc_temp), helpers.measure_z_edges(qc_temp)])
        loss_val += np.linalg.norm(label - train_classes[i])**2
    return loss_val/len(train_classes)

## Generate Quantum circuit

Create a quantum circuit as described in https://journals.aps.org/prxquantum/pdf/10.1103/PRXQuantum.4.010328.

In [ ]:
layers = 2
blocks = 2
qc_equivariant = helpers.make_full_circuit(layers, blocks, data_param_dict)
qc_equivariant.decompose().draw()

## Setup Optimizer 

We use the Qiskit implementation of the ADAM optimizer.

In [ ]:
opt = ADAM(maxiter=50,
               tol=1e-6,
               lr=0.1,
               beta_1=0.9,
               beta_2=0.99,
               noise_factor=1e-8,
               eps=1e-10,
               amsgrad=False,
               snapshot_dir=None)

start = 0.5 * np.pi * (np.random.rand(36) - 0.5)
print(start)

## Run minimization process
Warning: this takes a while. 

In [ ]:
res = opt.minimize(lambda x: loss(qc_equivariant, train_pos, train_labels, x), x0 = start, bounds = [-np.pi, np.pi])
print(helpers.score(qc_equivariant, res.x, test_pos, test_labels))

## Test with pre-obtained parameters

As the optimization loop is quite long to run, the following parameters are given. These are the result of one optimization process. They give around 72% accuracy on test dataset. 

In [ ]:
sample_params = [ 0.85088804, -0.16598115,  0.26359942,  0.562003  , -0.04098523,
                  0.62178465, -0.71733895,  0.04031313,  0.92661584,  0.21138858,
                 -0.65799891,  0.85270021,  0.50494867,  0.74347194, -0.66101966,
                  0.20141943,  0.29542234,  0.04507088, -0.45123592, -0.09142401,
                  0.96749934, -0.37687953, -0.11190734, -0.99035946,  0.58611021,
                 -0.49306862,  0.0736983 ,  0.31039856,  1.62303328, -0.02937282,
                  0.35112483, -1.82985848, -0.05467669,  1.06182326, -0.45204521,
                  0.18327106]

print(helpers.score(qc_equivariant, sample_params, test_pos, test_labels))

## Test a non-geometric QML model

For comparison purpose, define a non-geometric QML model. This model is agnostic to any symmetry. It has 36 parameters, just like the G-QML model. 

In [ ]:
def define_agnostic_circuit(data_param_dict):
    qc = QuantumCircuit(9)
    for j in range(9):
        qc.rx(2 * np.pi / 3 * data_param_dict[f'x_{j}'], j)
    qc.append(EfficientSU2(num_qubits = 9, reps = 1, parameter_prefix = 'theta'), range(9))
    return qc

qc_agnostic = define_agnostic_circuit(data_param_dict)
qc_agnostic.decompose().decompose().draw()

### Run minimization process
Warning: this also takes a while.

In [ ]:
res_agn = opt.minimize(lambda x: loss(qc_agnostic, train_pos, train_labels, x), x0 = start, bounds = [-np.pi, np.pi])
print(helpers.score(qc_agnostic, res_agn.x, test_pos, test_labels))

### Test with sample parameters

This is what I obtained as a result of 1 optimization. The accuracy is around 45% on test data, significantly worse than the geometric model

In [ ]:
sample_params_agn = [ 2.60820587, -2.36543919,  0.00241482,  0.70965185, -1.2097236 ,
        1.46766323, -1.00959514, -1.11436405, -1.20381658, -1.40004612,
        1.47004275, -1.50029131, -0.82709146,  0.41857608, -1.84802844,
       -1.37763997, -1.23584003, -1.23057168,  1.72378491, -2.31549326,
        1.66816684,  0.66558259,  1.15545305,  1.56933712,  1.78319254,
       -2.34117593,  2.07923172,  0.52225609, -0.39391978, -0.05000988,
       -0.7884391 , -0.28555215,  0.18436415, -1.25122078,  0.20366388,
        0.17484095]

print(helpers.score(qc_agnostic, sample_params_agn, test_pos, test_labels))